In [95]:
import torch
import numpy as np

In [96]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Input
x = torch.tensor([3.0])

In [97]:
# Forward pass
y_pred = w * x + b
print("y_pred:", {y_pred.item()})

y_pred: {7.0}


In [98]:
# Loss
y_true = torch.tensor([10.0])
loss = (y_pred - y_true) ** 2
print("Loss:", {loss.item()})

Loss: {9.0}


In [99]:
print("Avant backward: w.grad =", w.grad)

Avant backward: w.grad = None


In [100]:
loss.backward()

In [101]:
print("Après backward: w.grad =", w.grad)
print("Apres backward: b.grad =", b.grad)

Après backward: w.grad = tensor([-18.])
Apres backward: b.grad = tensor([-6.])


In [102]:
learning_rate = 0.01
with torch.no_grad(): # Désactiver le tracking des gradients pour l'update w = w + alpha * w
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [103]:
print("Nouveau poids : w =", w, "b =", b)

Nouveau poids : w = tensor([2.1800], requires_grad=True) b = tensor([1.0600], requires_grad=True)


In [104]:
# Réinitialiser les gradients
w.grad.zero_()
b.grad.zero_()
print("Après zero_(): w.grad =", w.grad, "b.grad =", b.grad)

Après zero_(): w.grad = tensor([0.]) b.grad = tensor([0.])


In [105]:
import torch.nn as nn

perceptron = nn.Linear(1, 1)

# Voir les poids initiaux
print("Poids :", perceptron.weight)
print("Biais :", perceptron.bias)

Poids : Parameter containing:
tensor([[-0.2722]], requires_grad=True)
Biais : Parameter containing:
tensor([-0.3021], requires_grad=True)


In [106]:
# Forward
x = torch.tensor([3.0]) # Shape (batch_size, features)
print(f"input : {x}")
y_pred = perceptron(x)
print(y_pred)


input : tensor([3.])
tensor([-1.1188], grad_fn=<ViewBackward0>)


In [107]:
# Loss
y_true = torch.tensor([10.0])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [108]:
loss

tensor(123.6270, grad_fn=<MseLossBackward0>)

In [109]:
# Backward
loss.backward()

In [110]:
print(f"Gradient de poids : {perceptron.weight.grad}")
print(f"Gradient de biais : {perceptron.bias.grad}")

Gradient de poids : tensor([[-66.7126]])
Gradient de biais : tensor([-22.2375])


In [111]:
import torch.optim as optim
optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [112]:
optimizer.step()

In [113]:
print("Nouveau poids :", perceptron.weight)
print("Nouveau biais :", perceptron.bias)

Nouveau poids : Parameter containing:
tensor([[0.3949]], requires_grad=True)
Nouveau biais : Parameter containing:
tensor([-0.0797], requires_grad=True)


In [114]:
optimizer.zero_grad()

In [115]:
print(f"Poids après zero_grad : {perceptron.weight.grad}")
print(f"Biais après zero_grad : {perceptron.bias.grad}")

Poids après zero_grad : None
Biais après zero_grad : None


In [116]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [117]:
model = SimpleNet()

In [118]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [119]:
x = torch.tensor([1.0, 2.0])
y = torch.tensor([5.0])
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)

loss.backward()

In [120]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import onnxruntime as rt
import matplotlib.pyplot as plt


In [121]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)



In [122]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_data = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [123]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [124]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits

In [125]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [126]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)

In [127]:
logits

tensor([[-0.0369,  0.0068, -0.0701,  0.0135, -0.1061, -0.0629,  0.0459,  0.0253,
         -0.0087,  0.1294]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [128]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0968, 0.1011, 0.0936, 0.1018, 0.0903, 0.0943, 0.1051, 0.1030, 0.0996,
         0.1143]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [129]:
y_pred = pred_probab.argmax(dim=1)
y_pred


tensor([9], device='cuda:0')

In [130]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [131]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5}]")


In [132]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [133]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


loss: 2.303890  [   64/60000]
loss: 2.293590  [ 6464/60000]
loss: 2.291224  [12864/60000]
loss: 2.284647  [19264/60000]
loss: 2.281651  [25664/60000]
loss: 2.286632  [32064/60000]
loss: 2.272811  [38464/60000]
loss: 2.279360  [44864/60000]
loss: 2.261609  [51264/60000]
loss: 2.263924  [57664/60000]
Test Error: 
 Accuracy: 36.5%, Avg loss: 2.258961 

Epoch 2
-------------------------------
loss: 2.268828  [   64/60000]
loss: 2.249856  [ 6464/60000]
loss: 2.251795  [12864/60000]
loss: 2.256744  [19264/60000]
loss: 2.237898  [25664/60000]
loss: 2.239049  [32064/60000]
loss: 2.239892  [38464/60000]
loss: 2.215648  [44864/60000]
loss: 2.225399  [51264/60000]
loss: 2.207261  [57664/60000]
Test Error: 
 Accuracy: 63.5%, Avg loss: 2.195612 

Epoch 3
-------------------------------
loss: 2.198908  [   64/60000]
loss: 2.206436  [ 6464/60000]
loss: 2.190112  [12864/60000]
loss: 2.181303  [19264/60000]
loss: 2.181831  [25664/60000]
loss: 2.154043  [32064/60000]
loss: 2.135504  [38464/60000]
loss: 

In [134]:
example_input = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
model.eval()
onnx_program = torch.onnx.export(model, example_input, input_names=["input"], output_names=["output"], dynamo=True)

[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [135]:
onnx_program.save("model.onnx")
sess = rt.InferenceSession("model.onnx", providers= rt.get_available_providers())
input_name = sess.get_inputs()[0].name
pred_onnx = sess.run(None, {input_name: example_input[0].numpy()})
print(pred_onnx)

[array([[-0.2821919 , -1.2141246 , -1.3455317 , -0.04980967,  0.9074416 ,
         0.41972417, -0.80220217,  2.288466  , -0.41802472,  1.4256362 ]],
      dtype=float32)]
